sql code for efi2 deficit for alcohol current status

In [2]:
 #this query iassigns drinking status to individuals in the care home cohort based on eficodes, it only identifies the most recent code to admission to the care home. i then did a union all with the current smoker equivalent
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_alcohol_current_status`as (

#this retrieves all snomedcodes for the care home cohort
with b as (
  SELECT person_id, first_episodestartdate
  from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`),

person as (
    SELECT 
  person_id,
  snomedcode,
  cast(numericvalue as numeric) as numericvalue_1,
  dateevent
  FROM`yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  WHERE person_id in(SELECT person_id FROM `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
# select all codes in ch cohort with harmful drinking recorded
 harm_all as (  
   SELECT 
    person.person_id,
    person.snomedcode,
    person.dateevent as harm_date,
    b.first_episodestartdate as ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit= 'Alcohol' AND Otherinstructions = 'Harmful drinking')),
#number all harmful drinking codes according to date per person and time constraint 5 or less years
  harm_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by harm_date desc) as harm_seq
  from harm_all
  Where harm_date < ch_entry_date AND date_diff(harm_all.ch_entry_date,harm_all.harm_date,year) <= 5),
#filter for the most recent code of harmful drinking 
  harm_filtered as (
    select 
    *,
    case when harm_date IS NOT NULL then 'harmful drinking' end as alcohol_status
    from harm_all_no
  where harm_seq = 1),
#retrieve all codes of harmful drinking for ch cohort
  high_risk_all as (
    SELECT 
    person.person_id,
    person.snomedcode,
    person.dateevent as high_risk_date,
    b.first_episodestartdate as ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit ='Alcohol' AND Otherinstructions = 'Higher risk drinking')),
#sequence codes according to date of entry per person and apply time constraint 
  high_risk_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by high_risk_date desc) as high_risk_seq
  from high_risk_all
  Where high_risk_date < ch_entry_date AND date_diff(high_risk_all.ch_entry_date,high_risk_all.high_risk_date,year) <= 5),
#filter for most recent code 
  high_risk_filtered as (
    select 
    *,
    case when high_risk_date IS NOT NULL then 'higher risk drinking' end as alcohol_status
    from high_risk_all_no
  where high_risk_seq = 1),
#retrieve all snomed codes in ch cohort for lower risk drinking and left join ch entry date from master table 
  low_risk_all as (
    SELECT 
    person.person_id,
    person.snomedcode,
    person.dateevent as low_risk_date,
    b.first_episodestartdate as ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit ='Alcohol' AND Otherinstructions = 'Lower risk drinking')),
#sequence codes according to date per person and apply time constraints
  low_risk_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by low_risk_date desc) as low_risk_seq
  from low_risk_all
  Where low_risk_date < ch_entry_date AND date_diff(low_risk_all.ch_entry_date,low_risk_all.low_risk_date
  ,year) <= 5),

  low_risk_filtered as (
    select 
    *,
     case when low_risk_date IS NOT NULL then 'lower risk drinking' end as alcohol_status
    from low_risk_all_no
  where low_risk_seq = 1),

  zero_all as (
    SELECT 
    person.person_id,
    person.snomedcode,
    person.dateevent as zero_date,
    b.first_episodestartdate as ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit ='Alcohol' AND Otherinstructions = 'Zero alcohol')),

  zero_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by zero_date desc) as zero_seq
  from zero_all
  Where zero_date < ch_entry_date AND date_diff(zero_all.ch_entry_date,zero_all.zero_date,year) < 6),

  zero_filtered as (
  SELECT
  *,
   case when zero_date IS NOT NULL then 'zero drinking' end as alcohol_status
  from zero_all_no
  where zero_seq = 1),

  unit_all as (
 SELECT 
    person.person_id,
    person.snomedcode,
    numericvalue_1,
    person.dateevent as unit_date,
    b.first_episodestartdate as ch_entry_date
    from person
    LEFT JOIN b
    on person.person_id = b.person_id
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where Deficit ='Alcohol' AND Otherinstructions ='Alcohol - numeric (0 = code as zero alcohol; 1-20 = code as lower risk drinking; 21-48 = code as higher risk drinking; 49+ = code as harmful drinking)')),

  unit_all_no as (
  Select 
  *,
  row_number() over (partition by person_id order by unit_date desc) as unit_seq
  from unit_all
  Where unit_date < ch_entry_date AND date_diff(unit_all.ch_entry_date,unit_all.unit_date,year) <= 5),


  unit_filtered as (
  SELECT
  person_id,
  snomedcode,
  unit_date,
  unit_all_no.ch_entry_date,
  unit_seq,
   case when unit_all_no.numericvalue_1 > 48 then 'harmful drinking'
   when unit_all_no.numericvalue_1 between 21 AND 48 then 'higher risk drinking'
   when unit_all_no.numericvalue_1 between 1 AND 20 then 'lower risk drinking'
   when unit_all_no.numericvalue_1 < 1 then 'zero drinking'
    end as alcohol_status
  from unit_all_no
  where unit_seq = 1),


  merged_drinking as (
  
  SELECT 
  * from harm_filtered

  UNION ALL 

  select * from zero_filtered

  UNION ALL 

  select * from low_risk_filtered
  
  UNION ALL 
  
  select * from unit_filtered),

merged_drinking_filtered as (
select
*,
row_number() over (partition by person_id order by harm_date desc) as alcohol_seq
from merged_drinking)


  select
    person_id,
    snomedcode,
    harm_date as snomed_date,
    ch_entry_date,
    alcohol_status,
    alcohol_seq
  from merged_drinking_filtered
  where alcohol_seq = 1
  order by person_id)





ERROR: Error in parse(text = x, srcfile = src): <text>:2:8: unexpected symbol
1:  #this query iassigns drinking status to individuals in the care home cohort based on eficodes, it only identifies the most recent code to admission to the care home. i then did a union all wi
2: CREATE OR
          ^
